<a href="https://colab.research.google.com/github/CMWENLIU/Drug_label_embedding/blob/master/drug_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip3 install --quiet wget
!pip install html2text

In [61]:
import os
import csv
import pandas as pd
import re
import wget
import html2text
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import nltk.data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
urls = ["https://raw.githubusercontent.com/CMWENLIU/Drug_label_embedding/master/fdalabel-query-1132.csv",
       "https://raw.githubusercontent.com/CMWENLIU/Drug_label_embedding/master/indications_humanrx.xlsx",
       ]
[wget.download(u) for u in urls]    

['fdalabel-query-1132 (1).csv', 'indications_humanrx (1).xlsx']

In [0]:
def clean_str(string):
   
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = string.replace("\\","")
    return string.strip().lower()

In [83]:
df = pd.read_csv('fdalabel-query-1132.csv')
df1 = df[['Most Recent SPL Date (YYYY/MM/DD)','Active Ingredient UNII(s)', 'SET ID']]
df1.rename(columns={'Most Recent SPL Date (YYYY/MM/DD)': 'SPL_Date', 'Active Ingredient UNII(s)': 'AIU', 'SET ID':'SET_ID'}, inplace=True)
df2 = pd.read_excel('indications_humanrx.xlsx')
df1.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,SPL_Date,AIU,SET_ID
0,20160429,0CD5FD6S2M; 5E8K9I0O4U; 6OZP39ZG8H,1a7e4806-3138-4b7a-b798-a99e163ee846
1,20170531,382L14738L,0ea37235-35fd-410d-b8c4-40ba15fe1294
2,20170803,04079A1RDZ; ZS7284E0ZP,7ea701ce-e7d3-4349-a9c2-642a501d45c8
3,20181121,3S252O2Z4X,203459a5-9769-44a0-8396-43d110207ba9
4,20190227,54Q18076QB; 839I73S42A,21335fe4-d395-4501-ac2a-2f20d7520da9


In [87]:
df1 = df1.groupby('AIU').first()
res_df = pd.merge(df1, df2, on='SET_ID', how='inner')
res_df.head()

,SPL_Date,SET_ID,CONTENT_XML
0,20180129,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,"<section xmlns=""urn:hl7-org:v3"" ID=""ID_e17ca14..."
1,20160520,2c272405-837d-007f-e054-00144ff88e88,"<section xmlns=""urn:hl7-org:v3"" ID=""id_link_2c..."
2,20110601,16a441d8-610e-de3f-c352-9c0fab0b9d3c,"<section xmlns=""urn:hl7-org:v3"" ID=""ID_8ebcf16..."
3,20110601,7f2a7a7b-d750-623a-74e0-f96b8d41cbb8,"<section xmlns=""urn:hl7-org:v3"" ID=""ID_a2e7cca..."
4,20110601,a9b53259-8388-76fa-6e83-2b8ca1f5358b,"<section xmlns=""urn:hl7-org:v3"" ID=""ID_72F7AFF..."


In [89]:
h = html2text.HTML2Text()
h.ignore_links = True

set_ids, xmls, texts, sentences,  = [], [], [], []
for idx, row in res_df.iterrows():
  v = str(res_df.loc[idx, 'CONTENT_XML'])
  if v:
    text = h.handle(v).replace("``", "")
    subs = sent_tokenize(text)
    for t in subs:
      if len(t) > 30:
        set_ids.append(res_df.loc[idx, 'SET_ID'])
        xmls.append(res_df.loc[idx, 'CONTENT_XML'])
        texts.append(text)
        sentences.append(clean_str(t))
final_df = pd.DataFrame()
final_df['set_ids'] = set_ids
#final_df['xmls'] = xmls
#final_df['texts'] = texts
final_df['sentences'] = sentences
final_df.to_csv('final.csv') 
final_df.head()

,set_ids,sentences
0,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,indications and usage minocycline hydrochlorid...
1,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,respiratory tract infections caused by mycopla...
2,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,lymphogranuloma venereum caused by chlamydia t...
3,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,psittacosis ( ornithosis ) due to chlamydia ps...
4,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,"trachoma caused by chlamydia trachomatis , alt..."
